# Garments Classification Model

## Import Requirements

In [ ]:
from fastai.vision.data import ImageDataLoaders
from fastai.vision.all import *
from fastai.imports import *
import fastbook as fb
import gc
import pandas as pd
import pickle
import numpy as np
from pathlib import Path

%matplotlib inline

## Import dataset

In [ ]:
list_cloth_path = "../Dataset/DeepFashion_DS/DeepFashion_DS_TextFiles/list_category_cloth.txt"
list_img_path = "../Dataset/DeepFashion_DS/DeepFashion_DS_TextFiles/list_category_img.txt"

df_cloth=pd.read_csv(list_cloth_path,skiprows=1,delim_whitespace=True)
df_img=pd.read_csv(list_img_path,skiprows=1,delim_whitespace=True)

In [ ]:
df_cloth.to_dict()
category_map = df_cloth
category_map["value"] = category_map.index + 1

In [ ]:
dict_category = category_map.set_index("value").to_dict()["category_name"]
dict_cloth = category_map.set_index("category_name").to_dict()["category_type"]

In [ ]:
df = df_img
df["category_label"] = df["category_label"].map(dict_category)
df["garment_type"] = df["category_label"].map(dict_cloth)

In [ ]:
df["image_name"] = df["image_name"].str.replace("img/","DeepFashion_DS/DeepFashion_DS_IMG/")

In [ ]:
df

In [ ]:
shoe_df = pd.DataFrame(columns = ['image_name', 'category_label'])

root = "../Dataset/EdgeNet_Shoe_DS/training/"
dirlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]

for category_name in dirlist:
    route = root + category_name + "/"
    
    directory = os.fsencode(route)

    for file in os.listdir(directory): #for cycle to iterate over folder
        filename = os.fsdecode(file) #get file name
        if filename.endswith(".jpg"): #if ends with .jpg
            shoe_df = shoe_df.append({'image_name' : str("EdgeNet_Shoe_DS/training/"+category_name+"/"+filename), 'category_label' : category_name},ignore_index = True)

In [ ]:
with open('../Dataset/DeepFashion_DS/DeepFashion_DS_TextFiles/train.txt') as f:
    train_list = f.read().splitlines()

train_list = [row.replace("img/","DeepFashion_DS/DeepFashion_DS_IMG/") for row in train_list]

cloth_df = df[df["image_name"].isin(train_list)].drop("garment_type",axis=1)

frames = [cloth_df,shoe_df]

complete_df = pd.concat(frames)

complete_df.to_csv("../Dataset/train_list.csv",index=False)

## Image Load

In [ ]:
PATH = "../Dataset/"
TRAINING_PATH = "train_list.csv"

data = ImageDataLoaders.from_csv(PATH, csv_fname=TRAINING_PATH,
                                 item_tfms=Resize(300),
                                 batch_tfms=aug_transforms(size=224, min_scale=0.9),
                                 valid_pct=0.1,
                                 splitter=RandomSplitter(seed=42), #seed=42
                                 num_workers=0)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
data.show_batch(max_n=7, nrows=1)

## Model Training

In [ ]:
learn = cnn_learner(data, resnet34, metrics=[accuracy,error_rate], pretrained=True)
learn.fine_tune(3)
learn.save('stage-1_resnet34')

In [ ]:
gc.collect()
torch.cuda.empty_cache()

### Learning rate finder

In [ ]:
learn = cnn_learner(data, resnet34, metrics=accuracy)
lr_steep = learn.lr_find()

In [ ]:
lr_steep

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
learn.fine_tune(8, base_lr=5e-3)

### Discriminative Learning Rates

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(6, lr_max=slice(1e-7, 1e-3))

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, lr_max=slice(1e-7, 1e-3))

### Export model

In [ ]:
#modelname = learn.model
#modelname.cpu()
#torch.save(modelname, '../Model/stage-1_resnet34.pkl')
learn.export("../Model/stage-1_resnet34.pkl")

## Evaluation

### Mappings

In [ ]:
category_dict = {
    "top":      1,
    "bottom":   2, 
    "shoes":    3,
    "onepiece": 4   # ignored in the MVP version
}

formality_dict = {
    "formality":3,
    "casual":2, 
    "sport":1
}

weather_dict = {
    "cold":3,
    "sunny":2, 
    "rainny":1
}

type_dict = {    # category, formality, weather
"Anorak" :        (1, 1, 3),
"Blazer":         (1, 3, 2),
"Blouse":         (1, 2, 2),
"Bomber" :        (1, 2, 3),
"Button-Down":    (1, 2, 2 ),
"Cardigan" :      (1, 2, 3),
"Flannel" :       (1, 2, 3),
"Halter":         (1,2,2),
"Henley" :        (1, 2,3),
"Hoodie" :        (1, 2, 3),
"Jacket" :        (1, 2, 3),
"Jersey"  :       (1, 1, 2),
"Parka"   :       (1,2,3),
"Peacoat" :       (1,3,3),
"Poncho" :        (1,2,3),
"Sweater"  :      (1,2,3),
"Tank" :          (1,3,2),
"Tee" :           (1,2,2),
"Top"  :          (1,2,2),
"Turtleneck" :    (1,3,3),
"Capris" :        (2,2,2),
"Chinos"  :       (2,2,2),
"Culottes"  :     (2,2,2),
"Cutoffs" :       (2,2,2),
"Gauchos"  :      (2,2,2),
"Jeans" :         (2,2,2),
"Jeggings" :      (2,2,2),
"Jodhpurs"  :     (2,1,2),
"Joggers"  :      (2,1,3),
"Leggings"  :     (2,1,2),
"Sarong"  :       (2,2,2),
"Shorts"  :       (2,2,2),
"Skirt"   :       (2, 2,2),
"Sweatpants" :    (2, 1, 3),
"Sweatshorts" :   (2, 1, 2 ),
"Trunks"       :  (2, 1, 2),
"Caftan"       :  (4,3,2),
"Cape"         :  (4,3,3),
"Coat"         :  (4, 3, 3),
"Coverup"      :  (4, 2, 2),
"Dress"        :  (4, 2, 2),
"Jumpsuit"     :  (4, 2, 2),
"Kaftan"        : (4, 3, 2),
"Kimono"         :(4, 3, 2),
"Nightdress"     :(4, 2, 2),
"Onesie"         :(4, 2, 2),
"Robe"           :(4, 2, 2),
"Romper"       :  (4, 2, 2),
"Shirtdress"    : (4, 2, 2),
"Sundress"     :  (4, 2, 2),
"boots":(3, 2, 3) ,
"flip_flops":(3, 2, 3),
"loafers":(3, 3, 2),
"sandals":(3, 2, 2),
"sneakers":(3, 2, 2),
"soccer_shoes":(3,1,2)}

In [ ]:
inv_category_dict = dict(zip(category_dict.values(), category_dict.keys()))
inv_formality_dict = dict(zip(formality_dict.values(), formality_dict.keys()))
inv_weather_dict = dict(zip(weather_dict.values(), weather_dict.keys()))

In [ ]:
type_df = pd.DataFrame(type_dict).T
type_df.columns = ['Garment_type', 'Formality', 'Weather']

In [ ]:
type_df["Garment_type"]=type_df["Garment_type"].map(inv_category_dict)
type_df["Formality"]=type_df["Formality"].map(inv_formality_dict)
type_df["Weather"]=type_df["Weather"].map(inv_weather_dict)

In [ ]:
type_df

### Predict

In [ ]:
pred_df = pd.DataFrame(columns=["image_name","category_label","pred_category_label","garment_type","formality","weather"])

In [ ]:
pred_df["image_name"] = complete_df["image_name"]
pred_df["category_label"] = complete_df["category_label"]

In [ ]:
pred_df = pred_df.drop(pred_df[pred_df["category_label"]=="Dress"].index)

In [ ]:
pred_df.reset_index().drop(["index"], axis=1)

In [ ]:
learn = load_learner('../Model/stage-1_resnet34.pkl')

In [ ]:
permuted_indices = np.random.permutation(len(pred_df))
N_PARTITIONS = 5
dfs = []
for i in range(N_PARTITIONS):
    dfs.append(pred_df.iloc[permuted_indices[i::N_PARTITIONS]])

In [ ]:
dfs[0]

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
cont = 1
avance = 1
for slice_pred_df in dfs:
    for image in slice_pred_df["image_name"]:
        image_path = "../Dataset/"+image
        slice_pred_df.loc[slice_pred_df["image_name"]==image,"pred_category_label"]=learn.predict(image_path)[0]
        gc.collect()
        torch.cuda.empty_cache()
        print(avance)
        avance+=1
    filepath = Path("Development/Predicted_DF/"+"pred_df"+str(cont)+".csv") 
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    slice_pred_df.to_csv(filepath,index=False)
    cont+=1
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
df1 = pd.read_csv("../Development/Predicted_DF/pred_df1.csv")
df2 = pd.read_csv("../Development/Predicted_DF/pred_df2.csv")
df3 = pd.read_csv("../Development/Predicted_DF/pred_df3.csv")
df4 = pd.read_csv("../Development/Predicted_DF/pred_df4.csv")
df5 = pd.read_csv("../Development/Predicted_DF/pred_df5.csv")

complete_pred_df = pd.concat([df1,df2,df3,df4,df5])

In [ ]:
complete_pred_df["garment_type"] = complete_pred_df["pred_category_label"].map(type_dict)
complete_pred_df

In [ ]:
def tuple_split(x, pos):
    return x[pos]
complete_pred_df['formality'] = complete_pred_df.apply(lambda x: tuple_split(x['garment_type'], 1),axis=1)
complete_pred_df['weather'] = complete_pred_df.apply(lambda x: tuple_split(x['garment_type'], 2),axis=1)
complete_pred_df['garment_type'] = complete_pred_df.apply(lambda x: tuple_split(x['garment_type'], 0),axis=1)


In [94]:
filepath = Path("Predicted_DF/complete_pred_df.csv") 
filepath.parent.mkdir(parents=True, exist_ok=True)  
complete_pred_df.to_csv(filepath,index=False)